## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Apr. 13th Apr. 20th


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,April-18-2025,Condo/Co-op,14 Colonial Club Dr #105,Boynton Beach,FL,33435.0,159900.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/14-Col...,Beaches MLS,RX-11075238,N,Y,26.502816,-80.058752


In [8]:
print('Input county name:')
county = input()

Input county name:
Palm Beach


In [9]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [10]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [11]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [12]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-04-18    32
2025-04-17    32
2025-04-15    31
2025-04-16    24
2025-04-14    19
2025-04-20     1
Name: count, dtype: int64

In [13]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [14]:
df_top_ten = df2.head(10)

In [15]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
126,PAST SALE,2025-04-15,Condo/Co-op,210 SE Mizner Blvd Ph 03,Boca Raton,FL,33432.0,10000000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/210-SE-Mi...,Beaches MLS,RX-11016958,N,Y,26.347009,-80.083249
9,PAST SALE,2025-04-15,Condo/Co-op,100 Sunrise Ave #504,Palm Beach,FL,33480.0,4100000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Beaches MLS,RX-11070770,N,Y,26.719562,-80.034847
60,PAST SALE,2025-04-18,Condo/Co-op,1500 S Ocean Blvd Unit S-1602,Boca Raton,FL,33432.0,2900000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1500-S-Oc...,Beaches MLS,RX-11061063,N,Y,26.330462,-80.073714
72,PAST SALE,2025-04-16,Condo/Co-op,2155 S Ocean Blvd #10,Delray Beach,FL,33483.0,2700000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2155-S-...,Beaches MLS,RX-11056722,N,Y,26.433506,-80.062813
64,PAST SALE,2025-04-15,Condo/Co-op,350 S Ocean Blvd Unit 4-D,Boca Raton,FL,33432.0,2350000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-11048259,N,Y,26.347273,-80.070212
87,PAST SALE,2025-04-14,Condo/Co-op,250 S Ocean Blvd Unit 9h,Boca Raton,FL,33432.0,2350000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/250-S-Oce...,Beaches MLS,RX-11044983,N,Y,26.348725,-80.069993
42,PAST SALE,2025-04-14,Condo/Co-op,2580 S Ocean Blvd Unit 1 B 6,Palm Beach,FL,33480.0,2000000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2580-S-Oc...,Beaches MLS,RX-11047683,N,Y,26.626708,-80.037346
48,PAST SALE,2025-04-16,Condo/Co-op,13244 Polo Club Rd Unit C105,Wellington,FL,33414.0,1795000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Wellington/13244-Pol...,Beaches MLS,RX-11055400,N,Y,26.642836,-80.253836
79,PAST SALE,2025-04-17,Condo/Co-op,223 Atlantic Ave Unit 4d,Palm Beach,FL,33480.0,1450000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/223-Atlan...,Beaches MLS,RX-11049466,N,Y,26.723706,-80.039057
57,PAST SALE,2025-04-17,Condo/Co-op,701 S Olive Ave #1101,West Palm Beach,FL,33401.0,1125000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-11062100,N,Y,26.706646,-80.052576


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [16]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [17]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [18]:
response_df = pd.DataFrame(response_list)

In [19]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [20]:
df_top_ten = merged_df

In [21]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-04-15,Condo/Co-op,210 SE Mizner Blvd Ph 03,Boca Raton,FL,33432.0,10000000.0,4.0,4.5,...,-80.083249,https://www.redfin.com/FL/Boca-Raton/210-SE-Mi...,Kathy Koch Pitlake,Douglas Elliman,None,None,Claude Champagne,Champagne & Parisi Real Estate,None,None
1,PAST SALE,2025-04-15,Condo/Co-op,100 Sunrise Ave #504,Palm Beach,FL,33480.0,4100000.0,2.0,2.5,...,-80.034847,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None
2,PAST SALE,2025-04-18,Condo/Co-op,1500 S Ocean Blvd Unit S-1602,Boca Raton,FL,33432.0,2900000.0,3.0,3.5,...,-80.073714,https://www.redfin.com/FL/Boca-Raton/1500-S-Oc...,Mark Howard Hansen,Coldwell Banker,None,None,Mark Howard Hansen,Coldwell Banker,None,None
3,PAST SALE,2025-04-16,Condo/Co-op,2155 S Ocean Blvd #10,Delray Beach,FL,33483.0,2700000.0,3.0,3.0,...,-80.062813,https://www.redfin.com/FL/Delray-Beach/2155-S-...,Rachelle Hirt-Beresh,Compass Florida LLC,Scott Gerow,Compass Florida LLC,Dustin Balmuth,Balmuth & Stone Real Estate LLC,None,None
4,PAST SALE,2025-04-15,Condo/Co-op,350 S Ocean Blvd Unit 4-D,Boca Raton,FL,33432.0,2350000.0,3.0,3.0,...,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Lauren Pines,Lang Realty/BR,None,None
5,PAST SALE,2025-04-14,Condo/Co-op,250 S Ocean Blvd Unit 9h,Boca Raton,FL,33432.0,2350000.0,2.0,2.5,...,-80.069993,https://www.redfin.com/FL/Boca-Raton/250-S-Oce...,Bonnie Heatzig,Compass Florida LLC,None,None,Sharon Kirkland,Illustrated Properties LLC,None,None
6,PAST SALE,2025-04-14,Condo/Co-op,2580 S Ocean Blvd Unit 1 B 6,Palm Beach,FL,33480.0,2000000.0,3.0,3.0,...,-80.037346,https://www.redfin.com/FL/Palm-Beach/2580-S-Oc...,Tim Frater,Frater Realty LLC,None,None,Tim Frater,Frater Realty LLC,None,None
7,PAST SALE,2025-04-16,Condo/Co-op,13244 Polo Club Rd Unit C105,Wellington,FL,33414.0,1795000.0,3.0,3.0,...,-80.253836,https://www.redfin.com/FL/Wellington/13244-Pol...,Roberta Feinberg,Illustrated Properties LLC (We,None,None,Matthew Johnson,EXP Realty LLC,None,None
8,PAST SALE,2025-04-17,Condo/Co-op,223 Atlantic Ave Unit 4d,Palm Beach,FL,33480.0,1450000.0,2.0,2.0,...,-80.039057,https://www.redfin.com/FL/Palm-Beach/223-Atlan...,Jean Heron,The Corcoran Group,Jason Haverkamp,William Raveis Real Estate,Robert Kalaghan,Illustrated Properties,None,None
9,PAST SALE,2025-04-17,Condo/Co-op,701 S Olive Ave #1101,West Palm Beach,FL,33401.0,1125000.0,2.0,2.0,...,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Christopher Cronican,Douglas Elliman,Lori Cronican,Douglas Elliman,Daniel Capozzi,Illustrated Properties LLC (Co,None,None


## Current Week's Values

In [22]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

139


In [23]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$507,079


In [24]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$321


In [25]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$70,484,000


In [26]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0          139              507079          321                     70484000.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [27]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
115
-----------
Input Previous Week Condo Average Sales Price:
481731
-----------
Input Previous Week Condo Average PSF:
329
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
55399100.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [29]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [28]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [29]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [30]:
pd.set_option('display.max_columns',None)

In [31]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-04-15,Condo/Co-op,210 SE Mizner Blvd Ph 03,Boca Raton,FL,33432.0,10000000.0,4.0,4.5,Alina Boca Raton Condo,5394.0,392000.0,2024.0,NaN,1854.0,9346.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/210-SE-Mi...,Beaches MLS,RX-11016958,N,Y,26.347009,-80.083249,https://www.redfin.com/FL/Boca-Raton/210-SE-Mi...,Kathy Koch Pitlake,Douglas Elliman,None,None,Claude Champagne,Champagne & Parisi Real Estate,None,None,1,orange


In [32]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [33]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-04-15,Condo/Co-op,210 SE Mizner Blvd Ph 03,Boca Raton,FL,33432.0,10000000.0,4.0,4.5,Alina Boca Raton Condo,5394.0,392000.0,2024.0,NaN,1854.0,9346.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/210-SE-Mi...,Beaches MLS,RX-11016958,N,Y,26.347009,-80.083249,https://www.redfin.com/FL/Boca-Raton/210-SE-Mi...,Kathy Koch Pitlake,Douglas Elliman,None,None,Claude Champagne,Champagne & Parisi Real Estate,None,None,1,orange
1,PAST SALE,2025-04-15,Condo/Co-op,100 Sunrise Ave #504,Palm Beach,FL,33480.0,4100000.0,2.0,2.5,One Hundred Sun & Surf Cond,1547.0,NaN,1977.0,NaN,2650.0,2600.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Beaches MLS,RX-11070770,N,Y,26.719562,-80.034847,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,2,blue
2,PAST SALE,2025-04-18,Condo/Co-op,1500 S Ocean Blvd Unit S-1602,Boca Raton,FL,33432.0,2900000.0,3.0,3.5,Addison,2792.0,NaN,1986.0,NaN,1039.0,4513.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1500-S-Oc...,Beaches MLS,RX-11061063,N,Y,26.330462,-80.073714,https://www.redfin.com/FL/Boca-Raton/1500-S-Oc...,Mark Howard Hansen,Coldwell Banker,None,None,Mark Howard Hansen,Coldwell Banker,None,None,3,blue
3,PAST SALE,2025-04-16,Condo/Co-op,2155 S Ocean Blvd #10,Delray Beach,FL,33483.0,2700000.0,3.0,3.0,Ocean Place-2155 Condo,4325.0,NaN,1981.0,NaN,624.0,2200.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2155-S-...,Beaches MLS,RX-11056722,N,Y,26.433506,-80.062813,https://www.redfin.com/FL/Delray-Beach/2155-S-...,Rachelle Hirt-Beresh,Compass Florida LLC,Scott Gerow,Compass Florida LLC,Dustin Balmuth,Balmuth & Stone Real Estate LLC,None,None,4,blue
4,PAST SALE,2025-04-15,Condo/Co-op,350 S Ocean Blvd Unit 4-D,Boca Raton,FL,33432.0,2350000.0,3.0,3.0,Beresford,2523.0,NaN,1976.0,NaN,931.0,4280.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-11048259,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Lauren Pines,Lang Realty/BR,None,None,5,blue
5,PAST SALE,2025-04-14,Condo/Co-op,250 S Ocean Blvd Unit 9h,Boca Raton,FL,33432.0,2350000.0,2.0,2.5,Marbella Condo,1670.0,180.0,1975.0,NaN,1407.0,2122.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/250-S-Oce...,Beaches MLS,RX-11044983,N,Y,26.348725,-80.069993,https://www.redfin.com/FL/Boca-Raton/250-S-Oce...,Bonnie Heatzig,Compass Florida LLC,None,None,Sharon Kirkland,Illustrated Properties LLC,None,None,6,blue
6,PAST SALE,2025-04-14,Condo/Co-op,2580 S Ocean Blvd Unit 1 B 6,Palm Beach,FL,33480.0,2000000.0,3.0,3.0,Palm Beach Stratford Condo,2266.0,NaN,1979.0,NaN,883.0,3621.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2580-S-Oc...,Beaches MLS,RX-11047683,N,Y,26.626708,-80.037346,https://www.redfin.com/FL/Palm-Beach/2580-S-Oc...,Tim Frater,Frater Realty LLC,None,None,Tim Frater,Frater Realty LLC,None,None,7,blue
7,PAST SALE,2025-04-16,Condo/Co-op,13244 Polo Club Rd Unit C105,Wellington,FL,33414.0,1795000.0,3.0,3.0,Meadow Brook Condo,2323.0,NaN,1981.0,NaN,773.0,1455.0,Sold,NaN,NaN,https://www.redfin.com/FL/Wellington/13244-Pol...,Beaches MLS,RX-11055400,N,Y,26.642836,-80.253836,https://www.redfin.com/FL/Wellington/13244-Pol...,Roberta Feinberg,Illustrated Properties LLC (We,None,None,Matthew Johnson,EXP Realty LLC,None,None,8,blue
8,PAST SALE,2025-04-17,Condo/Co-op,223 Atlantic Ave Unit 4d,Palm Beach,FL,33480.0,1450000.0,2.0,2.0,Palm Beac

In [34]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [35]:
m.save('index.html')

## Data snagger

In [36]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [37]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-04-15,Condo/Co-op,210 SE Mizner Blvd Ph 03,Boca Raton,FL,33432.0,10000000.0,4.0,4.5,Alina Boca Raton Condo,5394.0,392000.0,2024.0,NaN,1854.0,9346.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/210-SE-Mi...,Beaches MLS,RX-11016958,N,Y,26.347009,-80.083249,https://www.redfin.com/FL/Boca-Raton/210-SE-Mi...,Kathy Koch Pitlake,Douglas Elliman,None,None,Claude Champagne,Champagne & Parisi Real Estate,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [38]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_04212025


# CREATE TEMPLATE 

In [39]:
muni_set = set(df_top_ten['CITY'])

In [40]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [41]:
df_top_ten.reset_index(inplace=True,drop=True)

In [42]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [43]:
top_sale

'Alina Boca Raton Condo at 210 SE Mizner Blvd Ph 03 in Boca Raton'

In [44]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-04-15,Condo/Co-op,210 SE Mizner Blvd Ph 03,Boca Raton,FL,33432.0,10000000.0,4.0,4.5,Alina Boca Raton Condo,5394.0,392000.0,2024.0,NaN,1854.0,9346.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/210-SE-Mi...,Beaches MLS,RX-11016958,N,Y,26.347009,-80.083249,https://www.redfin.com/FL/Boca-Raton/210-SE-Mi...,Kathy Koch Pitlake,Douglas Elliman,None,None,Claude Champagne,Champagne & Parisi Real Estate,None,None,1,orange,Alina Boca Raton Condo at 210 SE Mizner Blvd P...
1,PAST SALE,2025-04-15,Condo/Co-op,100 Sunrise Ave #504,Palm Beach,FL,33480.0,4100000.0,2.0,2.5,One Hundred Sun & Surf Cond,1547.0,NaN,1977.0,NaN,2650.0,2600.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Beaches MLS,RX-11070770,N,Y,26.719562,-80.034847,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,2,blue,One Hundred Sun & Surf Cond at 100 Sunrise Ave...
2,PAST SALE,2025-04-18,Condo/Co-op,1500 S Ocean Blvd Unit S-1602,Boca Raton,FL,33432.0,2900000.0,3.0,3.5,Addison,2792.0,NaN,1986.0,NaN,1039.0,4513.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1500-S-Oc...,Beaches MLS,RX-11061063,N,Y,26.330462,-80.073714,https://www.redfin.com/FL/Boca-Raton/1500-S-Oc...,Mark Howard Hansen,Coldwell Banker,None,None,Mark Howard Hansen,Coldwell Banker,None,None,3,blue,Addison at 1500 S Ocean Blvd Unit S-1602 in Bo...
3,PAST SALE,2025-04-16,Condo/Co-op,2155 S Ocean Blvd #10,Delray Beach,FL,33483.0,2700000.0,3.0,3.0,Ocean Place-2155 Condo,4325.0,NaN,1981.0,NaN,624.0,2200.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2155-S-...,Beaches MLS,RX-11056722,N,Y,26.433506,-80.062813,https://www.redfin.com/FL/Delray-Beach/2155-S-...,Rachelle Hirt-Beresh,Compass Florida LLC,Scott Gerow,Compass Florida LLC,Dustin Balmuth,Balmuth & Stone Real Estate LLC,None,None,4,blue,Ocean Place-2155 Condo at 2155 S Ocean Blvd #1...
4,PAST SALE,2025-04-15,Condo/Co-op,350 S Ocean Blvd Unit 4-D,Boca Raton,FL,33432.0,2350000.0,3.0,3.0,Beresford,2523.0,NaN,1976.0,NaN,931.0,4280.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-11048259,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Lauren Pines,Lang Realty/BR,None,None,5,blue,Beresford at 350 S Ocean Blvd Unit 4-D in Boca...
5,PAST SALE,2025-04-14,Condo/Co-op,250 S Ocean Blvd Unit 9h,Boca Raton,FL,33432.0,2350000.0,2.0,2.5,Marbella Condo,1670.0,180.0,1975.0,NaN,1407.0,2122.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/250-S-Oce...,Beaches MLS,RX-11044983,N,Y,26.348725,-80.069993,https://www.redfin.com/FL/Boca-Raton/250-S-Oce...,Bonnie Heatzig,Compass Florida LLC,None,None,Sharon Kirkland,Illustrated Properties LLC,None,None,6,blue,Marbella Condo at 250 S Ocean Blvd Unit 9h in ...
6,PAST SALE,2025-04-14,Condo/Co-op,2580 S Ocean Blvd Unit 1 B 6,Palm Beach,FL,33480.0,2000000.0,3.0,3.0,Palm Beach Stratford Condo,2266.0,NaN,1979.0,NaN,883.0,3621.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2580-S-Oc...,Beaches MLS,RX-11047683,N,Y,26.626708,-80.037346,https://www.redfin.com/FL/Palm-Beach/2580-S-Oc...,Tim Frater,Frater Realty LLC,None,None,Tim Frater,Frater Realty LLC,None,None,7,blue,Palm Beach Stratford Condo at 2580 S Ocean Blv...
7,PAST SALE,2025-04-16,Condo/Co-op,13244 Polo Club Rd Unit C105,Wellington,FL,33414.0,1795000.0,3.0,3.0,Meadow Brook Condo,2323.0,NaN,1981.0,NaN,773.0,1455.0,Sold,NaN,NaN,http

In [47]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Alina Boca Raton Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $1,125,000 to $10,000,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, West Palm Beach, Palm Beach, Delray Beach, Wellington, Boca Raton, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 139 condo sales totaling $70,484,000 million from Apr. 13th to Apr. 20th. The previous week, brokers closed 115 condo sales totaling $55,399,100.

Last week’s units sold for an average of $507,079, higher than the $481,731 aver

In [48]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [49]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [50]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [51]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-04-15,Condo/Co-op,210 SE Mizner Blvd Ph 03,Boca Raton,FL,33432.0,10000000.0,4.0,4.5,Alina Boca Raton Condo,5394.0,392000.0,2024.0,NaN,1854.0,9346.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/210-SE-Mi...,Beaches MLS,RX-11016958,N,Y,26.347009,-80.083249,https://www.redfin.com/FL/Boca-Raton/210-SE-Mi...,Kathy Koch Pitlake,Douglas Elliman,None,None,Claude Champagne,Champagne & Parisi Real Estate,None,None,1,orange,Alina Boca Raton Condo at 210 SE Mizner Blvd P...
1,PAST SALE,2025-04-15,Condo/Co-op,100 Sunrise Ave #504,Palm Beach,FL,33480.0,4100000.0,2.0,2.5,One Hundred Sun & Surf Cond,1547.0,NaN,1977.0,NaN,2650.0,2600.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Beaches MLS,RX-11070770,N,Y,26.719562,-80.034847,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,2,blue,One Hundred Sun & Surf Cond at 100 Sunrise Ave...
2,PAST SALE,2025-04-18,Condo/Co-op,1500 S Ocean Blvd Unit S-1602,Boca Raton,FL,33432.0,2900000.0,3.0,3.5,Addison,2792.0,NaN,1986.0,NaN,1039.0,4513.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1500-S-Oc...,Beaches MLS,RX-11061063,N,Y,26.330462,-80.073714,https://www.redfin.com/FL/Boca-Raton/1500-S-Oc...,Mark Howard Hansen,Coldwell Banker,None,None,Mark Howard Hansen,Coldwell Banker,None,None,3,blue,Addison at 1500 S Ocean Blvd Unit S-1602 in Bo...
3,PAST SALE,2025-04-16,Condo/Co-op,2155 S Ocean Blvd #10,Delray Beach,FL,33483.0,2700000.0,3.0,3.0,Ocean Place-2155 Condo,4325.0,NaN,1981.0,NaN,624.0,2200.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2155-S-...,Beaches MLS,RX-11056722,N,Y,26.433506,-80.062813,https://www.redfin.com/FL/Delray-Beach/2155-S-...,Rachelle Hirt-Beresh,Compass Florida LLC,Scott Gerow,Compass Florida LLC,Dustin Balmuth,Balmuth & Stone Real Estate LLC,None,None,4,blue,Ocean Place-2155 Condo at 2155 S Ocean Blvd #1...
4,PAST SALE,2025-04-15,Condo/Co-op,350 S Ocean Blvd Unit 4-D,Boca Raton,FL,33432.0,2350000.0,3.0,3.0,Beresford,2523.0,NaN,1976.0,NaN,931.0,4280.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-11048259,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Lauren Pines,Lang Realty/BR,None,None,5,blue,Beresford at 350 S Ocean Blvd Unit 4-D in Boca...
5,PAST SALE,2025-04-14,Condo/Co-op,250 S Ocean Blvd Unit 9h,Boca Raton,FL,33432.0,2350000.0,2.0,2.5,Marbella Condo,1670.0,180.0,1975.0,NaN,1407.0,2122.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/250-S-Oce...,Beaches MLS,RX-11044983,N,Y,26.348725,-80.069993,https://www.redfin.com/FL/Boca-Raton/250-S-Oce...,Bonnie Heatzig,Compass Florida LLC,None,None,Sharon Kirkland,Illustrated Properties LLC,None,None,6,blue,Marbella Condo at 250 S Ocean Blvd Unit 9h in ...
6,PAST SALE,2025-04-14,Condo/Co-op,2580 S Ocean Blvd Unit 1 B 6,Palm Beach,FL,33480.0,2000000.0,3.0,3.0,Palm Beach Stratford Condo,2266.0,NaN,1979.0,NaN,883.0,3621.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2580-S-Oc...,Beaches MLS,RX-11047683,N,Y,26.626708,-80.037346,https://www.redfin.com/FL/Palm-Beach/2580-S-Oc...,Tim Frater,Frater Realty LLC,None,None,Tim Frater,Frater Realty LLC,None,None,7,blue,Palm Beach Stratford Condo at 2580 S Ocean Blv...
7,PAST SALE,2025-04-16,Condo/Co-op,13244 Polo Club Rd Unit C105,Wellington,FL,33414.0,1795000.0,3.0,3.0,Meadow Brook Condo,2323.0,NaN,1981.0,NaN,773.0,1455.0,Sold,NaN,NaN,http

In [52]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Alina Boca Raton Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $1,125,000 to $10,000,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, West Palm Beach, Palm Beach, Delray Beach, Wellington, Boca Raton, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 139 condo sales totaling $70,484,000 million from Apr. 13th to Apr. 20th. The previous week, brokers closed 115 condo sales totaling $55,399,100.

Last week’s units sold for an average of $507,079, higher than the $481,731 aver

In [53]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Boca-Raton/210-SE-Mizner-Blvd-33432/unit-03/home/192304189


In [54]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Palm-Beach/100-Sunrise-Ave-33480/unit-504/home/42430746


In [55]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/West-Palm-Beach/701-S-Olive-Ave-33401/unit-1101/home/42659745


In [56]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Palm-Beach/223-Atlantic-Ave-33480/unit-4d/home/194125376


In [57]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach/100-Sunrise-Ave-33480/unit-504/home/42430746


In [58]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Delray-Beach/2155-S-Ocean-Blvd-33483/unit-10/home/42581254


In [59]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-04-15,Condo/Co-op,210 SE Mizner Blvd Ph 03,Boca Raton,FL,33432.0,10000000.0,4.0,4.5,Alina Boca Raton Condo,5394.0,392000.0,2024.0,NaN,1854.0,9346.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/210-SE-Mi...,Beaches MLS,RX-11016958,N,Y,26.347009,-80.083249,https://www.redfin.com/FL/Boca-Raton/210-SE-Mi...,Kathy Koch Pitlake,Douglas Elliman,None,None,Claude Champagne,Champagne & Parisi Real Estate,None,None,1,orange,Alina Boca Raton Condo at 210 SE Mizner Blvd P...
1,PAST SALE,2025-04-15,Condo/Co-op,100 Sunrise Ave #504,Palm Beach,FL,33480.0,4100000.0,2.0,2.5,One Hundred Sun & Surf Cond,1547.0,NaN,1977.0,NaN,2650.0,2600.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Beaches MLS,RX-11070770,N,Y,26.719562,-80.034847,https://www.redfin.com/FL/Palm-Beach/100-Sunri...,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,2,blue,One Hundred Sun & Surf Cond at 100 Sunrise Ave...
2,PAST SALE,2025-04-18,Condo/Co-op,1500 S Ocean Blvd Unit S-1602,Boca Raton,FL,33432.0,2900000.0,3.0,3.5,Addison,2792.0,NaN,1986.0,NaN,1039.0,4513.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1500-S-Oc...,Beaches MLS,RX-11061063,N,Y,26.330462,-80.073714,https://www.redfin.com/FL/Boca-Raton/1500-S-Oc...,Mark Howard Hansen,Coldwell Banker,None,None,Mark Howard Hansen,Coldwell Banker,None,None,3,blue,Addison at 1500 S Ocean Blvd Unit S-1602 in Bo...
3,PAST SALE,2025-04-16,Condo/Co-op,2155 S Ocean Blvd #10,Delray Beach,FL,33483.0,2700000.0,3.0,3.0,Ocean Place-2155 Condo,4325.0,NaN,1981.0,NaN,624.0,2200.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2155-S-...,Beaches MLS,RX-11056722,N,Y,26.433506,-80.062813,https://www.redfin.com/FL/Delray-Beach/2155-S-...,Rachelle Hirt-Beresh,Compass Florida LLC,Scott Gerow,Compass Florida LLC,Dustin Balmuth,Balmuth & Stone Real Estate LLC,None,None,4,blue,Ocean Place-2155 Condo at 2155 S Ocean Blvd #1...
4,PAST SALE,2025-04-15,Condo/Co-op,350 S Ocean Blvd Unit 4-D,Boca Raton,FL,33432.0,2350000.0,3.0,3.0,Beresford,2523.0,NaN,1976.0,NaN,931.0,4280.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-11048259,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Lauren Pines,Lang Realty/BR,None,None,5,blue,Beresford at 350 S Ocean Blvd Unit 4-D in Boca...
5,PAST SALE,2025-04-14,Condo/Co-op,250 S Ocean Blvd Unit 9h,Boca Raton,FL,33432.0,2350000.0,2.0,2.5,Marbella Condo,1670.0,180.0,1975.0,NaN,1407.0,2122.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/250-S-Oce...,Beaches MLS,RX-11044983,N,Y,26.348725,-80.069993,https://www.redfin.com/FL/Boca-Raton/250-S-Oce...,Bonnie Heatzig,Compass Florida LLC,None,None,Sharon Kirkland,Illustrated Properties LLC,None,None,6,blue,Marbella Condo at 250 S Ocean Blvd Unit 9h in ...
6,PAST SALE,2025-04-14,Condo/Co-op,2580 S Ocean Blvd Unit 1 B 6,Palm Beach,FL,33480.0,2000000.0,3.0,3.0,Palm Beach Stratford Condo,2266.0,NaN,1979.0,NaN,883.0,3621.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2580-S-Oc...,Beaches MLS,RX-11047683,N,Y,26.626708,-80.037346,https://www.redfin.com/FL/Palm-Beach/2580-S-Oc...,Tim Frater,Frater Realty LLC,None,None,Tim Frater,Frater Realty LLC,None,None,7,blue,Palm Beach Stratford Condo at 2580 S Ocean Blv...
7,PAST SALE,2025-04-16,Condo/Co-op,13244 Polo Club Rd Unit C105,Wellington,FL,33414.0,1795000.0,3.0,3.0,Meadow Brook Condo,2323.0,NaN,1981.0,NaN,773.0,1455.0,Sold,NaN,NaN,http

## Time on Market Calculator

In [62]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 3, 12) ## List (Earlier) date
date2 = datetime(2025, 4, 15) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

34


## Clean CSV for Datawrapper Chart

In [63]:
chart_df = df_top_ten

In [64]:
chart_df = chart_df.fillna(" ")

In [65]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [66]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [67]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [68]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [69]:
chart_df['$/SQUARE FEET'].astype(float)

0    1854.0
1    2650.0
2    1039.0
3     624.0
4     931.0
5    1407.0
6     883.0
7     773.0
8    1522.0
9     676.0
Name: $/SQUARE FEET, dtype: float64

In [70]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [71]:
csv_date_string = today.strftime("%m_%d_%Y")

In [72]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [73]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-04-15,210 SE Mizner Blvd Ph 03 in Boca Raton,"$10,000,000",4,4.5,"5,394",2024,"$1,854",Beaches MLS,RX-11016958,Kathy Koch Pitlake Douglas Elliman,Claude Champagne Champagne & Parisi Real Estat...
1,2025-04-15,100 Sunrise Ave #504 in Palm Beach,"$4,100,000",2,2.5,"1,547",1977,"$2,650",Beaches MLS,RX-11070770,"Fern Fodiman Sotheby's Intl. Realty, Inc.","Fern Fodiman Sotheby's Intl. Realty, Inc."
2,2025-04-18,1500 S Ocean Blvd Unit S-1602 in Boca Raton,"$2,900,000",3,3.5,"2,792",1986,"$1,039",Beaches MLS,RX-11061063,Mark Howard Hansen Coldwell Banker,Mark Howard Hansen Coldwell Banker
3,2025-04-16,2155 S Ocean Blvd #10 in Delray Beach,"$2,700,000",3,3.0,"4,325",1981,$624,Beaches MLS,RX-11056722,Rachelle Hirt-Beresh Compass Florida LLC Scott...,Dustin Balmuth Balmuth & Stone Real Estate LLC...
4,2025-04-15,350 S Ocean Blvd Unit 4-D in Boca Raton,"$2,350,000",3,3.0,"2,523",1976,$931,Beaches MLS,RX-11048259,Michael Wells Premier Estate Properties Inc Sc...,Lauren Pines Lang Realty/BR
5,2025-04-14,250 S Ocean Blvd Unit 9h in Boca Raton,"$2,350,000",2,2.5,"1,670",1975,"$1,407",Beaches MLS,RX-11044983,Bonnie Heatzig Compass Florida LLC,Sharon Kirkland Illustrated Properties LLC
6,2025-04-14,2580 S Ocean Blvd Unit 1 B 6 in Palm Beach,"$2,000,000",3,3.0,"2,266",1979,$883,Beaches MLS,RX-11047683,Tim Frater Frater Realty LLC,Tim Frater Frater Realty LLC
7,2025-04-16,13244 Polo Club Rd Unit C105 in Wellington,"$1,795,000",3,3.0,"2,323",1981,$773,Beaches MLS,RX-11055400,Roberta Feinberg Illustrated Properties LLC (W...,Matthew Johnson EXP Realty LLC
8,2025-04-17,223 Atlantic Ave Unit 4d in Palm Beach,"$1,450,000",2,2.0,953,1979,"$1,522",Beaches MLS,RX-11049466,Jean Heron The Corcoran Group Jason Haverkamp ...,Robert Kalaghan Illustrated Properties
9,2025-04-17,701 S Olive Ave #1101 in West Palm Beach,"$1,125,000",2,2.0,"1,664",2008,$676,Beaches MLS,RX-11062100,Christopher Cronican Douglas Elliman Lori Cron...,Daniel Capozzi Illustrated Properties LLC (Co
